In [1]:
!pip install pennylane-qiskit

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 6.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.6/104.6 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 6.9 MB/s eta 0:00:00
  Created wheel for ibm-platform-services: filename=ibm_platform_services-0.49.0-py3-none-any.whl size=316177 sha256=3d400b4449505ba7464715f352d3017d740745b0a3792e5265ea451ebb80369f
  Stored in directory: /root/.cache/pip/wheels/23/95/32/bbd98f6f559d5851a5fd50ff5496ca9d6ac2c5dbf1ec2030dd
  Created wheel for ibm-cloud-sdk-core: filename=ibm_cloud_sdk_core-3.19.0-py3-none-any.whl size=94538 sha256=b719

A Very Simple pennylane circuit used for QNN

In [2]:
from qiskit.providers.fake_provider import FakeAuckland

#Quantum Noise Model Structure
import qiskit_aer.noise as noise

#Important Imports
import pennylane as qml
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from matplotlib import pyplot as plt
from pennylane import numpy as np
from multiprocessing import cpu_count
from keras.layers import *
from keras.models import Model
from keras.models import Sequential

#Noise Mitigation tools and circuit transpilation tools
from pennylane.transforms import mitigate_with_zne

#intialization
random_seed=965
np.random.seed(random_seed)
tf.random.set_seed(random_seed)
tf.keras.backend.set_floatx('float64')

(X_train,Y_train),(X_test,Y_test)=keras.datasets.mnist.load_data()


#Extracting Binary Data
t=[0,1]

binary_index=[i for i in range(len(Y_train)) if Y_train[i] in t]
X_train=X_train[binary_index]
Y_train=Y_train[binary_index]
binary_index=[i for i in range(len(Y_test)) if Y_test[i] in t]
X_test=X_test[binary_index]
Y_test=Y_test[binary_index]

#Place to save all
folder ="/content/sample_data/local_runtime/"


#Sample Size
n_train=1000
n_test=300

X_train=X_train[:n_train]
Y_train=Y_train[:n_train]
X_test=X_test[:n_test]
Y_test=Y_test[:n_test]
X_train=X_train/255*np.pi
X_test =X_test/255*np.pi
X_train = np.array(X_train[...,tf.newaxis],requires_grad=False,dtype=np.float64)
X_test  = np.array( X_test[...,tf.newaxis],requires_grad=False,dtype=np.float64)


#Parameters
n_epochs=30
batch_size=8

#System Details
print(cpu_count())
print(tf.config.list_physical_devices('GPU'))
print(tf.config.list_physical_devices('CPU'))


11490434/11490434 [==============================] - 1s 0us/step
2
[]
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [11]:
def QuantumLayer(provider,m,input_dim):

  weight={"weights":(2,m)}
  n=input_dim[1]

  noisemodel=noise.NoiseModel.from_backend(provider)

  dev=qml.device("qiskit.aer",wires=m,backend="aer_simulator_statevector",noise_model =noisemodel,shots=10)

  @qml.qnode(dev)
  def layer(inputs,weights):
      inputs=inputs[0]
      for i in range(m):

        qml.RY(inputs[n*i],wires=i)
        qml.RZ(inputs[n*i+1],wires=i)
        qml.RX(inputs[n*i+2],wires=i)
        qml.RY(inputs[n*i+3],wires=i)

      qml.BasicEntanglerLayers(weights,wires=range(m))
      return [qml.expval(qml.PauliZ(i)) for i in range(m)]

  return qml.qnn.KerasLayer(layer,weight,output_dim=m,)


In [ ]:
qlayer=QuantumLayer(FakeAuckland(),4,(4,4))

model = Sequential([
            keras.layers.Cropping2D(cropping=((2, 2), (2, 2))),
            keras.layers.AveragePooling2D(pool_size=(6,6),strides=(6, 6)),
            keras.layers.Flatten(),
            qlayer,
            keras.layers.Normalization(mean=0,variance=1,axis=None),
            keras.layers.Dense(1,activation="sigmoid"),
        ])


model.compile(optimizer="adam", loss = 'binary_crossentropy', metrics =['accuracy'])

hybrid = model.fit(X_train,
                    Y_train,
                    epochs = n_epochs,
                    batch_size = 1,
                    validation_data=(X_test,Y_test),
                    verbose=1,
                    shuffle = True,workers=cpu_count(), use_multiprocessing=True)
"""
w=model.get_weights()
model.set_weights(w)

model.evaluate(X,Y,verbose=1)
print(hybrid.history['accuracy'])
"""

Non-Linear Training

In [ ]:
from qiskit.providers.fake_provider import FakeLagosV2,FakeWashingtonV2,FakeGuadalupeV2

shape=X_test[0].shape

# Model
input=Input(shape=shape)
crop=Cropping2D(cropping=((2,2),(2,2)))(input)
pool=AveragePooling2D(pool_size=(3,3),strides=(3,3))(crop)
flat=Flatten()(pool)

img1,img2,img3,img4=tf.split(flat,[16,16,16,16],1)

qlayer1=QuantumLayer(FakeLagosV2()      ,2,(2,8))(img1)
qlayer2=QuantumLayer(FakeAuckland()     ,2,(2,8))(img2)
qlayer3=QuantumLayer(FakeWashingtonV2() ,2,(2,8))(img3)
qlayer4=QuantumLayer(FakeGuadalupeV2()  ,2,(2,8))(img4)

conc=tf.concat([qlayer1,qlayer2,qlayer3,qlayer4],axis=1)

norm=Normalization(mean=0,variance=1,axis=None)(conc)

dense=Dense(2,activation="softmax")(norm)

model=Model(inputs=input,outputs=dense)

model.compile(optimizer="adam", loss = 'sparse_categorical_crossentropy', metrics =['accuracy'])

hybrid = model.fit(X_train,
                    Y_train,
                    epochs = n_epochs,
                    batch_size = batch_size,
                    validation_data=(X_test,Y_test),
                    verbose=1,
                    shuffle = True,workers=cpu_count(), use_multiprocessing=True)


ZNE Implementation

In [ ]:
def QuantumLayerZNE(provider,m,input_dim):

  weight={"weights":(2,m)}
  n=input_dim[1]

  noisemodel=noise.NoiseModel.from_backend(provider)

  dev=qml.device("qiskit.aer",wires=m,backend="aer_simulator_statevector",noise_model =noisemodel,shots=10)

  @qml.qnode(dev)
  def layer(inputs,weights):
      inputs=inputs[0]
      for i in range(m):

        qml.RY(inputs[n*i],wires=i)
        qml.RZ(inputs[n*i+1],wires=i)
        qml.RX(inputs[n*i+2],wires=i)
        qml.RY(inputs[n*i+3],wires=i)

      qml.BasicEntanglerLayers(weights,wires=range(m))
      return [qml.expval(qml.PauliZ(i)) for i in range(m)]

  scale_factors = [1, 2, 3]

  layer = mitigate_with_zne(
        scale_factors=scale_factors,
        folding=qml.transforms.fold_global,
        extrapolate=qml.transforms.richardson_extrapolate,
    )(layer)
  return qml.qnn.KerasLayer(layer,weight,output_dim=m,)

# Rest of the working and usage is same

Add specific noise

In [3]:
custom_noise=noise.NoiseModel()
custom_noise.add_all_qubit_quantum_error(noise.pauli_error([('X', 0.7), ('I', 0.3)]),['cx', 'id', 'rz', 'x', 'sx','u1','u2','u3'])